In [7]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from datasets import Dataset, load_dataset
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.optim import Adam, lr_scheduler
from tqdm import tqdm
import evaluate
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import random
import pandas as pd
from collections import Counter
import pickle
import joblib
from copy import deepcopy


sns.set_style('whitegrid')

import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

violet = '#702b9d'
RANDOM_SEED = 1002

checkpoint = 'FacebookAI/xlm-roberta-base'
batch_size = 12 


In [ ]:
from transformers import XLMRobertaModel
from main_soft_align_swap import *



base_model = XLMRobertaModel.from_pretrained( checkpoint)
tokenizer = AutoTokenizer.from_pretrained( checkpoint)
Eval = namedtuple('Eval', ['predictions', 'label_ids'])

en_train = Dataset.from_file(os.getcwd() + '/data_en/en.train/data-00000-of-00001.arrow')
zh_train = Dataset.from_file(os.getcwd() + '/data_zh/zh.train/data-00000-of-00001.arrow')
    
zh_val = Dataset.from_file(os.getcwd() + '/data_zh/zh.dev/data-00000-of-00001.arrow')
para_dataset = deepcopy(en_train)
para_dataset = para_dataset.add_column("target_utt", zh_train['utt'])
para_dataset = para_dataset.add_column("target_slots", zh_train['slots_str'])
para_dataset = para_dataset.add_column("target_intents", zh_train['intent_str'])
para_dataset = para_dataset.map(lambda x: convert_train(x), batched=True)    

para_dataloader = DataLoader(para_dataset, batch_size= batch_size, shuffle=True, 
                            collate_fn=CollatorMASSIVEIntentClassSlotFill_para(tokenizer=tokenizer, max_length=512))
train_dataloader = DataLoader(en_train, batch_size= batch_size, shuffle=True, 
                            collate_fn=CollatorMASSIVEIntentClassSlotFill(tokenizer=tokenizer, max_length=512))
eval_dataloader = DataLoader(zh_val, batch_size= batch_size, shuffle=True,
                                collate_fn=CollatorMASSIVEIntentClassSlotFill(tokenizer=tokenizer, max_length=512))
train_eval_dataloader = DataLoader(zh_train, batch_size= batch_size, shuffle=True,
                                collate_fn=CollatorMASSIVEIntentClassSlotFill(tokenizer=tokenizer, max_length=512))
vocab = tokenizer.get_vocab()
vocab_size = len(vocab)

 # load mappings: should be using en mapping since order matters
with open(os.getcwd() + '/data_en/en.intents', 'r', encoding = 'UTF-8') as file:
    intent_labels_map = json.load(file)

with open(os.getcwd() + '/data_en/en.slots', 'r', encoding = 'UTF-8') as file:
    slot_labels_map = json.load(file)

with open(os.getcwd() + '/data_zh/zh.intents', 'r', encoding = 'UTF-8') as file:
    zh_intent_labels_map = json.load(file)

with open(os.getcwd() + '/data_zh/zh.slots', 'r', encoding = 'UTF-8') as file:
    zh_slot_labels_map =json.load(file)
    

base_model = XLMRobertaModel.from_pretrained('FacebookAI/xlm-roberta-base')
model = MultiTaskICSL(base_model, vocab_size, num_slot_labels=56, num_intents=60)


In [16]:
total_params  = sum(p.numel() for p in model.parameters() if p.requires_grad)
base_params = sum(p.numel() for p in base_model.parameters() if p.requires_grad)
base_params

278043648